So, first thing first. Ill start examining a radom point of New York, and see how dense is that area with a query to Foursquare API. The expected result is that New York its a totally hot area terribly expensive, and as such, not a safe bet for someone who is taking its first steps in the international scene. High risk high reward surely, but it has no meaning to throw yourself to the sharks like that, and ill try to prove it through data:

In [ ]:
#Import necesary libraries for the sake of this particular study:



In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [2]:
#Ill save the keys necesary to call the Foursquare API later, as shown in the course:
CLIENT_ID = '0ETFVTQEUWSRRP2EQ5JHZW0UUC4AMRXYX0YFFHQAKZLJJD10' # your Foursquare ID
CLIENT_SECRET = 'T4K2MZTXLDEVIPIL15LTDI3QPNBSHMYAIHXBCLNBSFFEH1FH' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 52
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0ETFVTQEUWSRRP2EQ5JHZW0UUC4AMRXYX0YFFHQAKZLJJD10
CLIENT_SECRET:T4K2MZTXLDEVIPIL15LTDI3QPNBSHMYAIHXBCLNBSFFEH1FH


In [3]:
#As a random point in a relevant area, i will reutilize the geolocator to the exact area as shown in the course:
address = '102 North End Ave, New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address, timeout=10)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7149555 -74.0153365


In [89]:
#Ill start the query using a ratio of 2500, expecting the result to surpass the 49 unfortunately limit of the API, using as keyword 'pizza':
search_query = 'pizza'
radius = 10000
print('done!')

done!


In [190]:
#Call the API:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=0ETFVTQEUWSRRP2EQ5JHZW0UUC4AMRXYX0YFFHQAKZLJJD10&client_secret=T4K2MZTXLDEVIPIL15LTDI3QPNBSHMYAIHXBCLNBSFFEH1FH&ll=40.36,-74.01&v=20180604&query=bank&radius=5000&limit=52'

In [197]:
#Request the information and display it properly:
results = requests.get(url).json()

In [192]:
#Normalize the info as shown over the course and transform it to a dataframe, displaying the first 3 results:
venues = results['response']['venues']
dataframe = json_normalize(venues)
dataframe.head(3)

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d13d941735', 'name': 'H...",NaN,NaN,NaN,NaN,NaN,NaN,False,4be16a84d816c928d5f7efd9,101 Ridge Rd,...,"[101 Ridge Rd, Little Silver, NJ 07739, United...","[{'label': 'display', 'lat': 40.35016004059454...",40.350160,-74.039729,NaN,07739,NJ,Red Bank Regional High School,v-1549514468,NaN
1,"[{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'S...",NaN,NaN,NaN,NaN,NaN,NaN,False,4b9a41e4f964a520c6a735e3,362 Broad St,...,"[362 Broad St, Red Bank, NJ 07701, United States]","[{'label': 'display', 'lat': 40.339588, 'lng':...",40.339588,-74.064127,NaN,07701,NJ,Super Foodtown of Red Bank,v-1549514468,209529457
2,"[{'id': '4bf58dd8d48988d129951735', 'name': 'T...",NaN,NaN,NaN,NaN,NaN,NaN,False,4af38fc5f964a5209fee21e3,175 Monmouth St.,...,"[175 Monmouth St., Red Bank, NJ 07701, United ...","[{'label': 'display', 'lat': 40.34828085719662...",40.348281,-74.074357,NaN,07701,NJ,NJT - Red Bank Station (NJCL),v-1549514468,NaN


In [201]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

As expected, the query surpass the 49 limits of the API, not good. Lets visualize the results, and later decrease some the radio to display more consistent results:

In [11]:
import pandas as pd
import folium
from folium.plugins import FastMarkerCluster, MarkerCluster

df = dataframe_filtered

folium_map = folium.Map(location=[latitude, longitude],
                        zoom_start=13.4,
                        tiles='Stamen Terrain')

mc = MarkerCluster(name="Marker Cluster")

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=10,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(folium_map)

mc.add_to(folium_map)

folium.LayerControl().add_to(folium_map)
folium_map

Surely its not for sale!
Lets decrease the radio and display the results again...

In [12]:
search_query = 'pizza'
radius = 1000
print('done!')

done!


In [17]:
import pandas as pd
import folium
from folium.plugins import FastMarkerCluster, MarkerCluster

df = dataframe_filtered

folium_map = folium.Map(location=[latitude, longitude],
                        zoom_start=13.4,
                        tiles='Stamen Terrain')

mc = MarkerCluster(name="Marker Cluster")

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=10,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(folium_map)

mc.add_to(folium_map)

folium.LayerControl().add_to(folium_map)
folium_map

49 results again, showing the overpopulation of places related to pizza, wont even imagine the scenario asking a more generic 'restaurants'

So, lets move around a little. Lets see in another corner of the country, say 7th avenue with 86th street, pretty close with lot of movement also. 

In [52]:
search_query = 'pizza'
radius = 5000
print('done!')

done!


In [23]:
latitude=40.62
longitude=-74.03
print("ready to go!")

ready to go!


In [200]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

I wont put all the steps again cause it is a pretty iterative process, and this notebook would get incredibly huge. But above is the result for the new query, 49 results again... lets see them.

In [29]:
import pandas as pd
import folium
from folium.plugins import FastMarkerCluster, MarkerCluster

df = dataframe_filtered

folium_map = folium.Map(location=[latitude, longitude],
                        zoom_start=13.4,
                        tiles='Stamen Terrain')

mc = MarkerCluster(name="Marker Cluster")

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=10,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(folium_map)

mc.add_to(folium_map)

folium.LayerControl().add_to(folium_map)
folium_map

Ok, totally "overpopulated" area. Lets move around a little. Unfortunately, this process would be much more efficient if the limit of results from Foursquare was not as it is now, but les go ahead. Lets move more to the south, working a little bit with google earth. 

In [30]:
latitude=40.45
longitude=-74.22
print("ready to go!")

ready to go!


In [199]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

In [43]:
import pandas as pd
import folium
from folium.plugins import FastMarkerCluster, MarkerCluster

df = dataframe_filtered

folium_map = folium.Map(location=[40.45, -74.22],
                        zoom_start=11.4,
                        tiles='Stamen Terrain')

mc = MarkerCluster(name="Marker Cluster")

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=10,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(folium_map)

mc.add_to(folium_map)

folium.LayerControl().add_to(folium_map)
folium_map

Ok, so 49 results again. A hot area again. Lets move again, to see if there is a less congested area 

In [58]:
latitude=40.31
longitude=-73.98
print("ready to go!")

ready to go!


In [64]:
import pandas as pd
import folium
from folium.plugins import FastMarkerCluster, MarkerCluster

df = dataframe_filtered

folium_map = folium.Map(location=[40.31, -73.98],
                        zoom_start=11.4,
                        tiles='Stamen Terrain')

mc = MarkerCluster(name="Marker Cluster")

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=10,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(folium_map)

mc.add_to(folium_map)

folium.LayerControl().add_to(folium_map)
folium_map

Wrong call again. Dont worry, this land somewhere. If one looks closely, the distribution is really dense there. Much less dispersion that in the other cases. So it seems that moving up to say Highlands or similar, can finally give a promising result. 

In [65]:
latitude=40.397
longitude=-73.986
print("ready to go!")

ready to go!


In [71]:
import pandas as pd
import folium
from folium.plugins import FastMarkerCluster, MarkerCluster

df = dataframe_filtered

folium_map = folium.Map(location=[40.397, -73.986],
                        zoom_start=12.4,
                        tiles='Stamen Terrain')

mc = MarkerCluster(name="Marker Cluster")

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=10,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(folium_map)

mc.add_to(folium_map)

folium.LayerControl().add_to(folium_map)
folium_map

So finally something. For a radio of 5000, just 13 results. We land in a Borough called Highlands, that seems promising. It is near the see, it doesnt seem to be on the desert, and its between really hot spots such as Long Branch as seen before, and Keyport -thee map that is just before Long Branch. It actually seems like Highlands have some movement after all, but Fair Haven, for example, has really not movements. Lets do the final map to see if moving a little bit toward Fair Heaven, reveals more information. Also, it would be nice to get the whole picture of the area. So lets zoom out a little more, and then we can proceed to move toward exploring the place founded.

In [88]:
latitude=40.36
longitude=-74.01
print("ready to go!")

ready to go!


In [96]:
import pandas as pd
import folium
from folium.plugins import FastMarkerCluster, MarkerCluster

df = dataframe_filtered

folium_map = folium.Map(location=[40.36, -74.01],
                        zoom_start=12,
                        tiles='Stamen Terrain')

mc = MarkerCluster(name="Marker Cluster")

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=10,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(folium_map)

mc.add_to(folium_map)

folium.LayerControl().add_to(folium_map)
folium_map

So, it seems that there are other hot spots, with high density, but Rumson Road remains with no movement at all. Finally, it seems, that this place is totally an option. At least if its not a ghost town or something. It seem that the untoched area is divided between Fair Haven and Rumson. Lets do some research about them.

In [97]:
from lxml import html
import requests

In [98]:
page = requests.get('https://en.wikipedia.org/wiki/Rumson,_New_Jersey')

In [99]:
tree=html.fromstring(page.content)

In [100]:
from bs4 import BeautifulSoup

In [102]:
soup = BeautifulSoup(page.text, 'html.parser')

In [141]:
import requests
from bs4 import BeautifulSoup
 
def count_words(url, the_word):
    r = requests.get(url, allow_redirects=False)
    soup = BeautifulSoup(r.content, 'lxml')
    words = soup.find(text=lambda text: text and the_word in text)
    print(words)
    return len(words)
 
def main():
    url = 'https://en.wikipedia.org/wiki/Rumson,_New_Jersey'
    word = 'income'
    count = count_words(url, word)
    print('\nUrl: {}\ncontains {} occurrences of word: {}'.format(url, count, word))
 
if __name__ == '__main__':
    main()

median household income

Url: https://en.wikipedia.org/wiki/Rumson,_New_Jersey
contains 23 occurrences of word: income


In [ ]:
So, les find manually the relevant words: median household income

In [ ]:
print(soup.prettify())

In [144]:
The median household income was of 134,281 which seems pretty high. Good news. 

In [149]:
import requests
from bs4 import BeautifulSoup
 
def main():
    url = 'https://en.wikipedia.org/wiki/Rumson,_New_Jersey'
    word = 'population'
    count = count_words(url, word)
    
if __name__ == '__main__':
    main()

, the borough's population was 7,122,


So there you go, a population of 7,122 for the community, not much but still close to other locations quite populated. So, lets dig a bit more. But first, we should look a bit more from the county where Rumson belongs, to see, at least, if it is not worst around...  

In [166]:
import requests
from bs4 import BeautifulSoup
 
def main():
    url = 'https://en.wikipedia.org/wiki/Monmouth_County,_New_Jersey'
    word = 'population was'
    count = count_words(url, word)
    
if __name__ == '__main__':
    main()

, the county's population was 626,351, making it the state's fifth-most populous county,


In [170]:
import requests
from bs4 import BeautifulSoup
 
def main():
    url = 'https://en.wikipedia.org/wiki/Monmouth_County,_New_Jersey'
    word = 'income'
    count = count_words(url, word)
    
if __name__ == '__main__':
    main()

per capita personal income


So, one must iterate the process to search for the key word, and see if there is any luck in the process. This systems does not always work, but is worth the try

In [171]:
page = requests.get('https://en.wikipedia.org/wiki/Monmouth_County,_New_Jersey')

In [172]:
soup = BeautifulSoup(page.text, 'html.parser')

In [ ]:
print(soup.prettify())

<a href="/wiki/Per_capita_personal_income_in_the_United_States" title="Per capita personal income in the United States">
        per capita personal income
       </a>
       of $69,410, the fifth-highest in New Jersey and ranked 74th of 3,113 counties in USA
       

So, not bad at all. The county its quite wealthy and Rumson income is even higher ($79,388 per capita). So, it is located in quite a financially healthy place. 

The next step its to work a little on the presentation to be shown to the italian friend. We seem to have a little of luck this time. But lets dig closer with more queries to the relevant spot, to understand more about its infraestructure.

In [177]:
#Ill start the query using a ratio of 2500, expecting the result to surpass the 49 unfortunately limit of the API, using as keyword 'pizza':
search_query = 'hospital'
radius = 5000
print('done!')

done!


In [181]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Fair Haven Animal Hospital,Veterinarian,NaN,US,Fair Haven,United States,2072,"[Fair Haven, NJ, United States]","[{'label': 'display', 'lat': 40.365799, 'lng':...",40.365799,-74.033221,NaN,NJ,514b3927e4b05ed6ecbc417e
1,Hospital For Wayward Children,None,15 Lisa Court,US,NaN,United States,4762,"[15 Lisa Court, New Jersey, United States]","[{'label': 'display', 'lat': 40.337375, 'lng':...",40.337375,-74.057638,NaN,New Jersey,4d143aedbb488cfa0c6695d4
2,Monmouth Medical Hospital,Shop & Service,NaN,US,NaN,United States,3171,"[New Jersey, United States]","[{'label': 'display', 'lat': 40.35266716169504...",40.352667,-73.973869,NaN,New Jersey,4e57ce9da8096558610db2e7
3,Riverview Medical Center,Hospital,1 Riverview Plz,US,Red Bank,United States,4635,"[1 Riverview Plz, Red Bank, NJ 07701, United S...","[{'label': 'display', 'lat': 40.35351484308679...",40.353515,-74.063975,07701,NJ,4b608208f964a5207bea29e3
4,Little Silver Animal Hospital,Veterinarian,NaN,US,Little Silver,United States,4491,"[Little Silver, NJ 07739, United States]","[{'label': 'display', 'lat': 40.32846459215351...",40.328465,-74.043031,07739,NJ,4d0b958b9a443704c09cba67
5,Two Rivers Vet,Veterinarian,457 River Rd,US,Fair Haven,United States,3184,"[457 River Rd, Fair Haven, NJ 07704, United St...","[{'label': 'display', 'lat': 40.35919951666666...",40.359200,-74.047529,07704,NJ,4cbb461ff50e224b81bbf8fb
6,Hospitalty Room #2 - MMC,Hospital,NaN,US,Long Branch,United States,6254,"[Long Branch, NJ 07740, United States]","[{'label': 'display', 'lat': 40.30462914823733...",40.304629,-73.997485,07740,NJ,4f0121b86da1cd035a0e9132
7,Riverview Hospital Dialysis,Medical Center,48 E Front St,US,Red Bank,United States,4596,"[48 E Front St, Red Bank, NJ 07701, United Sta...","[{'label': 'display', 'lat': 40.35266705430916...",40.352667,-74.063325,07701,NJ,4c6bee1b96eaa593007123df


In [182]:
import pandas as pd
import folium
from folium.plugins import FastMarkerCluster, MarkerCluster

df = dataframe_filtered

folium_map = folium.Map(location=[40.36, -74.01],
                        zoom_start=12,
                        tiles='Stamen Terrain')

mc = MarkerCluster(name="Marker Cluster")

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=10,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(folium_map)

mc.add_to(folium_map)

folium.LayerControl().add_to(folium_map)
folium_map

In [183]:
#Ill start the query using a ratio of 2500, expecting the result to surpass the 49 unfortunately limit of the API, using as keyword 'pizza':
search_query = 'school'
radius = 5000
print('done!')

done!


In [198]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]


In [188]:
import pandas as pd
import folium
from folium.plugins import FastMarkerCluster, MarkerCluster

df = dataframe_filtered

folium_map = folium.Map(location=[40.36, -74.01],
                        zoom_start=12,
                        tiles='Stamen Terrain')

mc = MarkerCluster(name="Marker Cluster")

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=10,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(folium_map)

mc.add_to(folium_map)

folium.LayerControl().add_to(folium_map)
folium_map

In [189]:
#Ill start the query using a ratio of 2500, expecting the result to surpass the 49 unfortunately limit of the API, using as keyword 'pizza':
search_query = 'bank'
radius = 5000
print('done!')

done!


So, lots of schools and hospital arounds. As seen before, sourrunded by very hot spots, and at the same time, with lots of potencial consumers with quite a wealthy situation. Seems like a safe bet. 